In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , classification_report , roc_auc_score
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
train = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")
test = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/test.csv")
sample = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/sample_submission.csv")

In [ ]:
train.drop('id' , axis=1 , inplace = True)

train.head()

In [ ]:
sns.distplot(train['Annual_Premium'])

In [ ]:
male_df = train[train['Gender'] == 'Male'].groupby('Age')['Vehicle_Damage'].count()
female_df = train[train['Gender'] == 'Female'].groupby('Age')['Vehicle_Damage'].count()

sns.lineplot(x = male_df.index , y = male_df.values , color='blue' , label = 'Male')
sns.lineplot(x = female_df.index , y = female_df.values , color='red' , label = 'Female')
plt.ylabel('Count')
plt.title('Age & Vehicle Damage')

In [ ]:
no_license_Male  = train[(train['Gender'] == 'Male') & (train['Driving_License'] == 0) & (train['Vehicle_Damage'] == 'Yes')].Gender.count()
no_license_Female  = train[(train['Gender'] == 'Female') & (train['Driving_License'] == 0) & (train['Vehicle_Damage'] == 'Yes')].Gender.count()

sns.barplot(x = ['Male' , 'Female'] , y = (no_license_Male , no_license_Female))
plt.title("People have no driving license and have vehicle damage")
plt.ylabel('Count')

In [ ]:
Male_annual = train[train['Gender'] == 'Male'].groupby('Age').Annual_Premium.mean()
Female_annual = train[train['Gender'] == 'Female'].groupby('Age').Annual_Premium.mean()


plt.figure(figsize = (13,5))
sns.lineplot(x = Male_annual.index , y = Male_annual.values , color='blue' , label = 'Male')
sns.lineplot(x = Female_annual.index , y = Female_annual.values , color='red' , label = 'Female')
plt.ylabel('Count')
plt.title('Age & Annual Premium')

In [ ]:
# Vehicle_Age problem

lto = []
def lessThanOne(x):
    if x == "< 1 Year":
        lto.append(1)
    else:
        lto.append(0)
        
boat = []        
def betweenOneAndTwo(x):
    if x == "1-2 Year":
        boat.append(1)
    else:
        boat.append(0)
        
gtt = []
def greaterThanTwo(x):
    if x == '> 2 Years':
        gtt.append(1)
    else:
        gtt.append(0)
    
    
        
train["Vehicle_Age"].apply(lessThanOne)
train["Vehicle_Age"].apply(betweenOneAndTwo)
train["Vehicle_Age"].apply(greaterThanTwo)


train['lessThanOne'] = lto
train['betweenOneAndTwo'] = boat
train['greaterThanTwo'] = gtt

In [ ]:
train = pd.get_dummies(train , columns = ['Gender' , 'Driving_License' , 'Previously_Insured' , 'Vehicle_Damage' , 'lessThanOne' , 'betweenOneAndTwo' ,'greaterThanTwo'])

In [ ]:
train.drop('Vehicle_Age' , axis = 1 , inplace = True)

In [ ]:
y = train.Response.values
x = train.drop('Response' , axis = 1)

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.30 , random_state = 0)

In [ ]:
model = RandomForestClassifier()
model.fit(x_train , y_train)

In [ ]:
print(accuracy_score(y_test , model.predict(x_test)))